In [ ]:
from transformers import pipeline
model_sentiment = pipeline("sentiment-analysis", model="climatebert/distilroberta-base-climate-detector",device=0)
#model_sentiment = pipeline("sentiment-analysis",device=0)

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_parquet("test-climate.parquet")

In [ ]:
df

,text,label
0,Sustainable strategy ‘red lines’ For our susta...,1
1,"Verizon’s environmental, health and safety man...",1
2,"In 2019, the Company closed a series of transa...",1
3,"In December 2020, the AUC approved the Electri...",1
4,"Finally, there is a reputational risk linked t...",1
...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",1
396,Climate change is producing changes in weather...,1
397,A sound and certain regulatory and fiscal envi...,0
398,"Across our global workforce, 20% of Gold Field...",0


In [ ]:
ctext = "text"
sent_col = "prediction"

In [ ]:
model_sentiment("climate change")

[{'label': 'yes', 'score': 0.9949465394020081}]

In [ ]:
df[sent_col] = "NEGATIVE"
df[sent_col+"_VALUE"] = 0
i = 0
sent = []
s_val = []
for s in df[ctext].to_list():
    #print(s)
    i+=1
    if i%100 ==0:
        print(i,end=" ")
    if len(s)>1:
        if(len(s)>2000):
            s=s[:2000]
        result = model_sentiment(s)[0]
        if result['label'].upper() == "NO":
            sent.append(0)
            s_val.append(-result['score'])
        else:
            sent.append(1)
            s_val.append(result['score'])
    else:
        sent.append(0)
        s_val.append(0)
df[sent_col] = sent
df[sent_col+"_VALUE"] = s_val

c:\ProgramData\Anaconda3\envs\trans\lib\site-packages\transformers\pipelines\base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


100 200 300 400 

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))


(0.9494554379506059, 0.9578125, 0.9535585742482295, None)
[[ 75   5]
 [  7 313]]
              precision    recall  f1-score   support

           0       0.91      0.94      0.93        80
           1       0.98      0.98      0.98       320

    accuracy                           0.97       400
   macro avg       0.95      0.96      0.95       400
weighted avg       0.97      0.97      0.97       400



In [ ]:
df

,text,label,prediction,prediction_VALUE
0,Sustainable strategy ‘red lines’ For our susta...,1,1,0.997672
1,"Verizon’s environmental, health and safety man...",1,1,0.996563
2,"In 2019, the Company closed a series of transa...",1,1,0.996962
3,"In December 2020, the AUC approved the Electri...",1,1,0.885795
4,"Finally, there is a reputational risk linked t...",1,1,0.998267
...,...,...,...,...
395,"In 2020, Banco do Brasil Foundation celebrated...",1,0,-0.991874
396,Climate change is producing changes in weather...,1,1,0.998372
397,A sound and certain regulatory and fiscal envi...,0,0,-0.951647
398,"Across our global workforce, 20% of Gold Field...",0,0,-0.954695


In [ ]:
import openai
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
with open('c:\\data\\openAI-key-g.txt', 'r') as file:
    key = file.readline()
openai.api_key = key

In [ ]:
import json
import time
def batch_gpt(prompt,target_texts, batch_size):
    l = len(target_texts)
    size = int(l / batch_size) + 1
    size = batch_size
    text_list = np.array_split(target_texts, 3)
    print(f"Total records {l}, number of chunks = {size}")
    rez_keys = []
    rez_vals = []
    rez = []
    for i, texts in enumerate(text_list):
        text = "\n".join(texts)
        p = prompt + text
        print("prompt", p)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": p}]
        )
        r = response["choices"][0]["message"]["content"]
        print(result, r)
        rez.append(r)
        try: 
            dictData= json.loads(r)
            keys = list(dictData.keys())
            values = list(dictData.values())
            rez_keys += keys
            rez_vals += values
        except:
            print("error parsing"+r)
        print(f"i={i + 1}: shape:{len(rez_keys)}")
    return rez_keys, rez_vals, rez
def batch_gpt_len(prompt,target_texts, batch_size):
    l = len(target_texts)
    rez_keys = []
    rez_vals = []
    rez = []
    i =0
    while i < l:
        text = ""
        for j in range(batch_size):
            if i<l:
                if batch_size > 1:
                    text += f"{i}. "+target_texts[i]+"\n\n"
                else:
                    text += target_texts[i]
            i += 1
        p = prompt + text
        #print(f"Prompt ({len(p)}):", p)
        # print(i, len(p))

        # try the API call and if it fails, wait 10 seconds and retry again (max 3 times)
        for j in range(6):
            try:
                response = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "user", "content": p}]
                )
                break
            except:
                print("error calling API, retrying...")
                time.sleep(10)
        
        r = response["choices"][0]["message"]["content"]
        rez.append(r)
        # convert r to int
        try:
            b = int(r[:1])
        except:
            b = -1
            print("error parsing"+r)
        print(i, len(p), "-", r, b, end=": ")
        # if i % 10 print 
        if i % 10 == 0:
            print()
        rez_keys.append(b)
    return rez,rez_keys

In [ ]:
texts = df["text"].to_list()
prompt = 'Is the following text about environment? Answer only with 1 if yes or 0 if not. \n\n'
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

1 311 - 1 1: 2 769 - 1 1: 3 698 - 0 0: 4 1011 - 0 0: 5 501 - 1 1: 6 470 - 1 1: 7 467 - 1 1: 8 515 - 0 0: 9 611 - 1 1: 10 386 - 1 1: 
11 675 - 0 0: 12 342 - 1 1: 13 354 - 1 1: 14 588 - 0 0: 15 308 - 1 1: 16 517 - 0 0: 17 367 - 1 1: 18 528 - 1 1: 19 451 - 1 1: 20 282 - 1 1: 
21 758 - 1 1: 22 517 - 0 0: 23 793 - 1 (about environment) 1: 24 738 - 1 1: 25 693 - 1 1: 26 444 - 1 1: 27 393 - 1 1: 28 333 - 1 1: 29 564 - 1 1: 30 696 - 1 1: 
31 694 - 1 1: 32 291 - 1 1: 33 390 - 1 1: 34 585 - 1 1: 35 749 - 1 1: 36 919 - 1 1: 37 674 - 1 1: 38 416 - 1 1: 39 589 - 1 1: 40 406 - 1 1: 
41 506 - 1 1: 42 384 - 1 1: 43 246 - 1 1: 44 661 - 1 1: 45 540 - 1 1: 46 332 - 1 1: 47 379 - 1 1: 48 639 - 1 1: 49 636 - 1 1: 50 350 - 0 0: 
51 311 - 1 1: 52 300 - 1 1: 53 537 - 1 1: 54 406 - 1 1: 55 1229 - 1 1: 56 540 - 1 1: 57 504 - 0 0: 58 316 - 1 1: 59 435 - 0 0: 60 461 - 1 1: 
61 373 - 1 1: 62 701 - 1 1: 63 453 - 0 0: 64 1474 - 1 1: 65 299 - 1 1: 66 1060 - 1 1: 67 434 - 0 0: 68 337 - 1 1: 69 514 - 1 1: 70 618 - 1 1:

In [ ]:

prompt = 'You are the sustainability, environment, and climate change expert. Is the following text about sustainability, the environment, or climate change? Answer only with 1 if the text is sustainability, enviroment or climate change related or 0 if not: \n\n'
rez, rez_keys = batch_gpt_len(prompt, texts, 1)

error calling API, retrying...
1 479 - 1 (Sustainability) 1: 2 937 - 1 (Environment and sustainability) 1: 3 866 - 0 0: 4 1179 - 0 0: 5 669 - 1 (Climate Change) 1: 6 638 - 1 (environment and sustainability related) 1: 7 635 - 1 (climate change related) 1: 8 683 - 1 (sustainability) 1: 9 779 - 1 (climate change) 1: 10 554 - 1 (climate change related) 1: 
11 843 - 0 0: 12 510 - 1 (climate change) 1: 13 522 - 1 (climate change related) 1: 14 756 - 0 0: 15 476 - 1 (environment related) 1: 16 685 - 0 0: 17 535 - 1 (sustainability related) 1: 18 696 - 1 (climate change) 1: 19 619 - 1 (Climate change related) 1: 20 450 - 1 (climate change related) 1: 
21 926 - 1 (environment related) 1: 22 685 - 0 0: 23 961 - 1 (climate change) 1: 24 906 - 1 (climate change) 1: 25 861 - 1 (climate change related) 1: 26 612 - 1 (Climate Change) 1: 27 561 - 1 (sustainability related) 1: 28 501 - 1 (climate change related) 1: 29 732 - 1 (climate change) 1: 30 864 - 1 (climate change related) 1: 
31 862 - 1 (envi

In [ ]:
texts = df["text"].to_list()
# prompt = 'You are the climate change expert. Is this text climate change related? Answer only with 1 if the text is climate change related or 0 if not and explain the answer: \n\n'
# prompt = 'You are the sustainability and climate change expert. Is the following about sustainability, enviroment or climate change? Answer only with 1 if the text is sustainability, enviroment or climate change related or 0 if not: \n\n'
rez, rez_keys = batch_gpt_len(prompt, texts, 1)


1 454 - 1 (Sustainability) 1: 2 912 - 1 (environment and sustainability) 1: 3 841 - 0 (not related to sustainability, environment or climate change) 0: 4 1154 - 0 0: 5 644 - 1 (climate change related) 1: 6 613 - 1 (sustainability, environment, and climate change) 1: 7 610 - 1 (climate change related) 1: 8 658 - 1 (sustainability) 1: 9 754 - 1 (climate change related) 1: 10 529 - 1 (climate change related) 1: 
11 818 - 0 0: 12 485 - 1 1: 13 497 - 1 (climate change and sustainability related) 1: 14 731 - 0 0: error parsingEnvironment: 1
15 451 - Environment: 1 -1: 16 660 - 0 0: 17 510 - 1 (sustainability and energy conservation initiatives) 1: 18 671 - 1 (climate change) 1: 19 594 - 1 (climate change) 1: 20 425 - 1 (climate change related) 1: 
21 901 - 1 (environment-related) 1: 22 660 - 0 0: 23 936 - 1 (climate change and sustainability) 1: 24 881 - 1 1: 25 836 - 1 (climate change related) 1: 26 587 - 1 (climate change related) 1: 27 536 - 1 1: 28 476 - 1 1: 29 707 - 1 (climate change r

In [ ]:
text = ["Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.",
"Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of facilities worldwide. The goal of these assessments is to identify and correct site-specific issues, and to educate and empower facility managers and supervisors to implement corrective actions. Verizon’s environment, health and safety efforts are directed and supported by experienced experts around the world that support our operations and facilities.",
"In 2019, the Company closed a series of transactions related to the sale of its Canadian fossil fuel-based electricity generation business. A transaction with Heartland Generation Ltd., an affiliate of Energy Capital Partners, included the sale of 10 partly or fully owned natural gas-fired and coal-fired electricity generation assets located in Alberta and British Columbia. In two other separate transactions, the Company sold its 50 per cent ownership interest in the Cory Cogeneration Station to SaskPower International and its 50 per cent ownership interest in Brighton Beach Power to Ontario Power Generation."]
p = prompt + "\n\n".join(text)
print("prompt", p)
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": p}]
)
r = response["choices"][0]["message"]["content"]
print(r)

prompt I will give you 10 sentences. You need to detect if the sentence is climate-related and return the result "1" if the sentence is climate related or "0" if the sentence is not climate-related.The output need to be in JSON fromat conaining the original sentence and the result {"original sentence":"result 1 or 0"} without any other explanations. 
Example of the output format: {"The climate is changing":1, "The skay is blue":0}

Sustainable strategy ‘red lines’ For our sustainable strategy range, we incorporate a series of proprietary ‘red lines’ in order to ensure the poorest- performing companies from an ESG perspective are not eligible for investment.
Verizon’s environmental, health and safety management system provides a framework for identifying, controlling, and reducing the risks associated with the environments in which we operate. Besides regular management system assessments, internal and third-party compliance audits and inspections are performed annually at hundreds of f

In [ ]:
col_rez = "gpt_rez-e"
col_label = "gpt_label-e"
df[col_rez ] = rez
df[col_label] = rez_keys

In [ ]:
df.to_csv("chat_gpt_climate3.csv")

In [ ]:
df[col_rez ].unique()

array(['1', '0', '1 (about environment)', '1 (Yes)',
       'There is a mention of environmental risk factors, so the answer is 1.',
       '1 (yes)', '1.', 'and Social and Relationship Capital. \n\n1',
       'No.'], dtype=object)

In [ ]:
# select rows that have gpt_label not 1 or 0
df[df[col_label] != 1][df[col_label] != 0]

C:\Users\dimit\AppData\Local\Temp\ipykernel_30608\2609781461.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df[col_label] != 1][df[col_label] != 0]


,text,label,prediction,prediction_VALUE,gpt_rez,gpt_label,gpt_rez-s,gpt_label-s,gpt_rez-c,gpt_label-c,gpt_rez-e,gpt_label-e


In [ ]:
# update the gpt_label-e to 0 where gpt_rez-e is No.
df.loc[df[col_rez] == "No.", col_label] = 0

In [ ]:
# count the the different values in the gpt_label column
df[col_label].value_counts()


gpt_label-c
 0    203
 1    196
-1      1
Name: count, dtype: int64

In [ ]:
# replace the gpt_label wiht 1 whre the gpt_label is not -1
df.loc[df[col_label] == 2, col_label] = 1
df.loc[df[col_label] == -1, col_label] = 1

In [ ]:
# calculate the precision and f1 score for df columns label and prediction
from sklearn.metrics import precision_recall_fscore_support
sent_col = col_label
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.784890010818608, 0.8703125, 0.8120742306788818, None)
[[ 71   9]
 [ 47 273]]
              precision    recall  f1-score   support

           0       0.60      0.89      0.72        80
           1       0.97      0.85      0.91       320

    accuracy                           0.86       400
   macro avg       0.78      0.87      0.81       400
weighted avg       0.89      0.86      0.87       400



In [ ]:
# create a new column with the gpt_label-all that is 1 if any of the gpt_label-e, gpt_label-s, gpt_label-c is 1
df["gpt_label-all"] = df["gpt_label-e"] + df["gpt_label-s"] + df["gpt_label-c"]
df.loc[df["gpt_label-all"] > 0, "gpt_label-all"] = 1


In [ ]:
sent_col = "gpt_label-s"
print(precision_recall_fscore_support(df['label'], df[sent_col], average='macro'))

# confusion matrix
from sklearn.metrics import confusion_matrix
print(confusion_matrix(df['label'], df[sent_col]))

# performnce report

from sklearn.metrics import classification_report
print(classification_report(df['label'], df[sent_col]))

(0.7357046826282216, 0.821875, 0.7571223860296797, None)
[[ 67  13]
 [ 62 258]]
              precision    recall  f1-score   support

           0       0.52      0.84      0.64        80
           1       0.95      0.81      0.87       320

    accuracy                           0.81       400
   macro avg       0.74      0.82      0.76       400
weighted avg       0.87      0.81      0.83       400

